In [43]:
import xml.etree.cElementTree as ET
import glob
import pandas as pd

In [53]:
musicxml_files = glob.glob('Data/*/*.xml')

In [54]:
musicxml_files[0]

'Data/Op.2412__A_Feeling_of_a_Static_Dove/score.xml'

In [60]:
accompany_dict = {} ### {(note_pitch, note_type): {(note_pitch, note_type): frequency}}
co_occurance_dict = {}

In [61]:
for musicxml_file in musicxml_files:
#     print(musicxml_file)
    note_count = 0
    tree = ET.parse(musicxml_file)
    root = tree.getroot()  
    ## get measure ##
    measures = root.findall('./part/measure')   
    for measure in measures:
        have_occur_x_dict = {} ### {x_position: (note_pitch, note_type)} ###
        previous_pitch_and_type = ()
        for note in measure.findall('note'):
            ### rest note ###
            if('default-x' not in note.attrib):
                previous_pitch_and_type = ()
                continue
            x_position = note.attrib['default-x']
        
            
            if(note.find('./unpitched') is not None): ## like drum 
                continue
            note_type = note.find('type').text
            note_count += 1
            
#             if(note.find('./pitch/step') is  None):
#                 print(note)
#                 print(note_count)
            note_pitch = note.find('./pitch/step').text + note.find('./pitch/octave').text
            if note.find('./pitch/alter') is not None:
                if note.find('./pitch/alter').text == '-1':
                    note_pitch = note_pitch[0] + 'b' + note_pitch[1]
                else:
                    note_pitch = note_pitch[0] + '#' + note_pitch[1]
            ### to add accompany pair if there have two note in same x_position ###
            if(x_position in have_occur_x_dict.keys()):
                if(have_occur_x_dict[x_position] not in accompany_dict):
                    accompany_dict[have_occur_x_dict[x_position]] = {(note_pitch, note_type):1}
                else:
                    if((note_pitch, note_type) in accompany_dict[have_occur_x_dict[x_position]]):
                        accompany_dict[have_occur_x_dict[x_position]][(note_pitch, note_type)] += 1
                    else:
                        accompany_dict[have_occur_x_dict[x_position]][(note_pitch, note_type)] = 1
            else:
                have_occur_x_dict[x_position] = (note_pitch, note_type)
            ### add co-occurance note ###
            if(len(previous_pitch_and_type) != 0):
                if(previous_pitch_and_type not in co_occurance_dict):
                    co_occurance_dict[previous_pitch_and_type] = {(note_pitch, note_type):1}
                else:    
                    if((note_pitch, note_type) in co_occurance_dict[previous_pitch_and_type]):
                        co_occurance_dict[previous_pitch_and_type][(note_pitch, note_type)] += 1
                    else:
                        co_occurance_dict[previous_pitch_and_type][(note_pitch, note_type)] = 1
            previous_pitch_and_type = (note_pitch, note_type)
        
    

## save as pickle


In [63]:
import pickle

In [67]:
accompany_file = open('accompany_dict.pickle','wb')
pickle.dump(accompany_dict,accompany_file)
accompany_file.close()

In [68]:
co_occurance_file = open('co_occurance_dict.pickle','wb')
pickle.dump(co_occurance_dict,co_occurance_file)
co_occurance_file.close()

### see table

In [1]:
from music21 import *
import music21
import copy
import pickle
import random

In [2]:
accompany_file = open('accompany_dict.pickle','rb')
accompany_dict = pickle.load(accompany_file)
accompany_file.close()

co_occurance_file = open('co_occurance_dict.pickle','rb')
co_occurance_dict = pickle.load(co_occurance_file)
co_occurance_file.close()

In [6]:
import pandas as pd

coocc_df = pd.read_pickle('accompany_dict.pickle')
df = pd.DataFrame.from_dict(coocc_df)

B4      C5   G4     F#4      E4      C4      D4   E4     A4  \
            quarter quarter half quarter quarter quarter quarter half eighth   
A#0 16th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
A#1 16th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    eighth      NaN     NaN  NaN     1.0     NaN     NaN     NaN  NaN    1.0   
    half        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    quarter     NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   

                 G4  ...     A7   B#4   F#6   A#2   D#3    B0     E#2  C#1  \
            quarter  ... eighth whole whole whole whole whole quarter 32nd   
A#0 16th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
A#1 16th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    eighth      NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    half        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    quarter     NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   

               A7  E#2  
            whole 16th  
A#0 16th      NaN  NaN  
A#1 16th      NaN  NaN  
    eighth    NaN  NaN  
    half      NaN  NaN  
    quarter   NaN  NaN  

[5 rows x 1043 columns]

In [19]:
df[100:120]

B4      C5   G4     F#4      E4      C4      D4   E4     A4  \
            quarter quarter half quarter quarter quarter quarter half eighth   
Ab1 16th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    32nd        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    64th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    eighth      NaN     3.0  NaN     NaN     NaN     NaN     1.0  NaN    NaN   
    half        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    quarter     NaN     7.0  NaN     NaN     2.0     6.0     1.0  NaN    NaN   
    whole       NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
Ab2 128th       NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    16th        NaN     NaN  NaN     NaN     NaN     1.0     1.0  NaN    1.0   
    32nd        NaN     NaN  NaN     NaN     NaN     2.0     NaN  NaN    NaN   
    64th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    eighth      NaN    25.0  NaN     NaN     NaN     6.0     7.0  NaN    3.0   
    half        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    quarter     2.0    20.0  NaN     3.0     1.0    29.0     8.0  NaN    NaN   
    whole       NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
Ab3 128th       NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    16th        NaN     NaN  NaN     NaN     NaN     1.0     NaN  NaN    NaN   
    32nd        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    64th        NaN     NaN  NaN     NaN     NaN     NaN     NaN  NaN    NaN   
    eighth      2.0    14.0  NaN     NaN     NaN     5.0     7.0  NaN    3.0   

                 G4  ...     A7   B#4   F#6   A#2   D#3    B0     E#2  C#1  \
            quarter  ... eighth whole whole whole whole whole quarter 32nd   
Ab1 16th        8.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    32nd        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    64th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    eighth      2.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    half        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    quarter     7.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    whole       NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
Ab2 128th       NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    16th        1.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    32nd        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    64th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    eighth      2.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    half        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    quarter    15.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    whole       NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
Ab3 128th       NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    16th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    32nd        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    64th        NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   
    eighth      4.0  ...    NaN   NaN   NaN   NaN   NaN   NaN     NaN  NaN   

               A7  E#2  
            whole 16th  
Ab1 16th      NaN  NaN  
    32nd      NaN  NaN  
    64th      NaN  NaN  
    eighth    NaN  NaN  
    half      NaN  NaN  
    quarter   NaN  NaN  
    whole     NaN  NaN  
Ab2 128th     NaN  NaN  
    16th      NaN  NaN  
    32nd      NaN  NaN  
    64th      NaN  NaN  
    eighth    NaN  NaN  
    half      NaN  NaN  
    quarter   NaN  NaN  
    whole     NaN  NaN  
Ab3 128th     NaN  NaN  
    16th      NaN  NaN  
    32nd      NaN  NaN  
    64th      NaN  NaN  
    eighth    